In [1]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
raw_scores = pd.read_excel('catan.xlsx', parse_dates=[0])

In [3]:
raw_scores.columns

Index([                'Oyun No',                   'Tarih',
                   'Asıl oyuncu',                    'Sıra',
                  'Yedek oyuncu',                 'Kazanan',
                          'Puan',                    'Renk',
                  'Şehir Sayısı',              'Köy Sayısı',
                    'Yol Sayısı',    'Gelişim Kartı Sayısı',
             'ilk Gelişim Kartı',          'İlk Şehirleşen',
                       'İlk Köy',            'Liman Sayısı',
       'İlk Hangi Kaynak yanına',                    'Kasa',
                          'Süre',               'Extension',
                'Başlangıç Saat',             'En Uzun Yol',
                       'Şövalye',     'İlk 2-1 Tuğla Liman',
            'İlk 2-1 Kaya Liman',     'İlk 2-1 Saman Liman',
           'İlk 2-1 Koyun Liman',      'İlk 2-1 Odun Liman',
                 'İlk 3-1 Liman',               '2-1 Tuğla',
                     '2-1 Saman',       2022-03-01 00:00:00,
                      '2

In [4]:
raw_scores.columns = ["game_id", "date", "player", "rank", "substitute_player", "winner", "score", "color",
                        "num_of_cities", "num_of_settlement", "num_of_roads", "num_of_dev_cards", 
                      "buy_dev_card_first", "builds_city_first", "builds_settlement_first", "num_of_harbor",
                     "which_resources_first", "cashier", "time_in_min", "is_extension",
                     "start_time", "longest_road", "largest_army", "first_brick_harbor",
                     "first_ore_harbor", "first_grain_harbor", "first_wool_harbor", "first_lumber_harbor",
                     "first_3to1_harbor", "num_of_brick_harbor", "num_of_grain_harbor", "num_of_3to1_harbor",
                     "num_of_lumber_harbor", "num_of_ore_harbor", "num_of_wool_harbor", "initial_dices","dessert_placement"]

In [5]:
raw_scores["player"].replace({"Batuhan": "B",
                             "Alperen": "A",
                             "Feyza": "F",
                             "Ecem": "E",
                             "Canberk": "Ck",
                             "Volkan": "V",
                             "Çağatay": "Cc",
                             "Melis": "M",
                             "İzel": "I",
                             "Aysima": "Ak"}, inplace=True)

raw_scores["substitute_player"].replace({"Batuhan": "B",
                             "Alperen": "A",
                             "Feyza": "F",
                             "Ecem": "E",
                             "Canberk": "Ck",
                             "Volkan": "V",
                             "Çağatay": "Cc",
                             "Melis": "M",
                             "İzel": "I",
                            "Aysima": "Ak"}, inplace=True)

raw_scores["winner"].replace({"Batuhan": "B",
                             "Alperen": "A",
                             "Feyza": "F",
                             "Ecem": "E",
                             "Canberk": "Ck",
                             "Volkan": "V",
                             "Çağatay": "Cc",
                             "Melis": "M",
                             "İzel": "I",
                             "Aysima": "Ak"}, inplace=True)


raw_scores["color"].replace({"Mavi": "Blue",
                             "Kırmızı": "Red",
                             "Yeşil": "Green",
                             "Kahverengi": "Brown",
                             "Beyaz": "White",
                             "Turuncu": "Orange"}, inplace=True)



raw_scores["dessert_placement"].replace({"Dış Çember": "Outer Circle",
                             "Orta Çember": "Middle",
                             "İç Çember": "Inner Circle"
                             }, inplace=True)

In [6]:
d = {"Tuğla": "Brick",
                                             "Kaya": "Ore",
                                             "Saman": "Grain",
                                             "Odun": "Lumber",
                                             "Koyun": "Wool"}

In [7]:
full = []
for i, row in raw_scores.iterrows():
    temp = []
    for j in row.which_resources_first.split(","):
        j = j.strip()
        temp.append(d[j])
    full.append(temp)
raw_scores["which_resources_first"] = full

In [8]:
raw_scores

,game_id,date,player,rank,substitute_player,winner,score,color,num_of_cities,num_of_settlement,...,first_lumber_harbor,first_3to1_harbor,num_of_brick_harbor,num_of_grain_harbor,num_of_3to1_harbor,num_of_lumber_harbor,num_of_ore_harbor,num_of_wool_harbor,initial_dices,dessert_placement
0,1.0,2022-02-22,B,1.0,NaN,A,7.0,Blue,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2022-02-22,A,2.0,NaN,A,10.0,White,5.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,2022-02-22,V,4.0,Cc,A,8.0,Red,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,2022-02-22,M,3.0,E,A,7.0,Orange,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,2022-03-06,A,2.0,NaN,Cc,5.0,White,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,53.0,2022-10-12,A,4.0,NaN,B,8.0,White,4.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,"5, 8, 10, 4, 9, 11",Middle
222,54.0,2022-11-12,V,1.0,NaN,B,9.0,Blue,3.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,"5, 10, 6, 11, 4",Middle
223,54.0,2022-11-12,E,2.0,NaN,B,9.0,Orange,1.0,4.0,...,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,"5, 6, 11, 3, 8",Middle
224,54.0,2022-11-12,B,3.0,NaN,B,10.0,Red,4.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"5, 9, 10, 4, 9, 11",Middle


In [9]:
raw_scores["substitute_player"].unique()

array([nan, 'Cc', 'E', 'F'], dtype=object)

In [10]:
print(raw_scores["color"].unique())
print(raw_scores["player"].unique())
print(raw_scores["player"].unique())
print(raw_scores["dessert_placement"].unique())

['Blue' 'White' 'Red' 'Orange' 'Green' 'Brown']
['B' 'A' 'V' 'M' 'F' 'Cc' 'E' 'Ck' 'Ak' 'I']
['B' 'A' 'V' 'M' 'F' 'Cc' 'E' 'Ck' 'Ak' 'I']
[nan 'Outer Circle' 'Middle']


In [11]:
raw_scores.to_csv("./catan_scores.csv", index=False)